In [117]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import string 

import nltk 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import demoji 



In [118]:
demoji.download_codes()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20092\2299328559.py:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [119]:
post_df = pd.read_csv('cleaned_posts.csv')
post_df.head()

,postID,originalContent,date,link,sentiment,totalLikes,totalReplies,replyToPostID,taggedSymbols,username,userid,totalImages,totalSymbols
0,29104030,Vkl luôn,2024-11-06T21:39:23.267+07:00,NaN,0,1,0,NaN,"[{'symb': '^DJI', 'price': 43496.43}]",Hoàng,5ededf24-12f7-41d9-b390-08ff631fc275,0,1
1,29104026,Đ.ịt cụ thằng Khải Trần hô VNI sập về 900 lần ...,2024-11-06T21:39:05.22+07:00,NaN,0,0,0,NaN,[],Datbg10,73778567-ec61-43eb-b3a0-b4a651b8bd3f,0,0
2,29104024,Tăng 1 phát bằng vn làm hai mấy năm :)),2024-11-06T21:39:04.077+07:00,NaN,0,12,2,NaN,"[{'symb': '^DJI', 'price': 43506.86}]",Trung Tuyến,5d597f38-3b24-4f40-952c-2b3f9be8e7d5,0,1
3,29104021,Má đáng full tiền...đau,2024-11-06T21:38:52.277+07:00,NaN,0,0,1,NaN,"[{'symb': 'VNINDEX', 'price': 1261.28}]",Duc Nguyen,225659c6-cf18-4e93-aa39-a294bae5b784,1,1
4,29104019,Gap khủng long,2024-11-06T21:38:49.99+07:00,NaN,0,2,0,NaN,"[{'symb': '^DJI', 'price': 43499.54}]",BINH NHI,da8ebfa2-9cd5-4dbf-84cd-9567f694f681,0,1


In [120]:
stop_words = pd.read_csv('vietnamese-stopwords.csv')
stop_words = stop_words['word'].tolist()
stop_words = set(stop_words)

def preprocess_text_optimized(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return ' '.join(word for word in text.split() if word not in stop_words)

In [121]:
post_content = post_df[['originalContent', 'sentiment']]
post_content['sentiment'] = post_content['sentiment'].apply(lambda x: 'positive' if x == 1 
                                                                                else 'negative' if x == -1 else 'neutral')

post_content['originalContent'] = post_content['originalContent'].apply(preprocess_text_optimized)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20092\1350555011.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_content['sentiment'] = post_content['sentiment'].apply(lambda x: 'positive' if x == 1
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20092\1350555011.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_content['originalContent'] = post_content['originalContent'].apply(preprocess_text_optimized)


In [122]:
# remove contents contains link only
post_content = post_content[~post_content['originalContent'].str.match(r'^\s*(http|https|www\.).*$', na=False)]
post_content.dropna(inplace=True)


In [123]:
post_content.head(1)

,originalContent,sentiment
0,vkl,neutral


In [124]:
cursed_words = pd.read_csv('Vietnamese_cursed_words.txt')
cursed_words = cursed_words['word'].tolist()
cursed_words = set(cursed_words)

def contains_cursed_words(text):
    words = text.lower().split()
    # return number of cursed words in the text
    number_of_cursed_words = sum(1 for word in words if word in cursed_words)
    if number_of_cursed_words > 0:
        return 1, number_of_cursed_words
    return 0, 0

# test function
contains_cursed_words('đệch đéo')
    

(1, 2)

In [125]:
post_content[['contains_cursed_words', 'numberOfCursedWords']] = post_content['originalContent'].apply(
    contains_cursed_words
).apply(pd.Series)

post_content

,originalContent,sentiment,contains_cursed_words,numberOfCursedWords
0,vkl,neutral,1,1
1,địt cụ thằng khải trần hô vni sập 900 100,neutral,1,1
2,1 phát vn hai mấy,neutral,0,0
3,má full tiềnđau,neutral,0,0
4,gap khủng long,neutral,0,0
...,...,...,...,...
272974,vc,neutral,0,0
272975,chết dập chết dụi hic,neutral,1,2
272976,múc rũ đỉnh,neutral,0,0
272977,đừng mong giá rẻ mấy ní đỏ ko mua xanh line,neutral,0,0


In [126]:
post_content['contains_cursed_words']
post_content[post_content['numberOfCursedWords'] > 1].head(5)

,originalContent,sentiment,contains_cursed_words,numberOfCursedWords
80,vãi lồn,neutral,1,2
116,ae đc xác thằng long quan official trôi dạt kh...,neutral,1,2
193,đọc mãi báo đéo công văn quyết định thu hồi đé...,negative,1,4
344,vre xác 100 thoát vin xác 1 7 vol 35tr tham gi...,neutral,1,3
358,trump đắc cử tổng thống 47 mỹ chứng khoán việt...,neutral,1,2


In [127]:
import scipy.sparse
from sklearn.feature_extraction.text import TfidfVectorizer

post_content_for_model = post_content[post_content['sentiment'] != 'neutral']

vectorizer = TfidfVectorizer()
X_text = vectorizer.fit_transform(post_content_for_model['originalContent'])
Y = post_content_for_model['sentiment']

X_train, X_test, Y_train, Y_test = train_test_split(X_text, Y, test_size = 0.2, stratify = Y, random_state = 42)

X = scipy.sparse.hstack([X_text, scipy.sparse.csr_matrix(post_content_for_model[['contains_cursed_words', 'numberOfCursedWords']].values)])


In [128]:
clf = RandomForestClassifier(n_jobs = -1)
clf.fit(X_train, Y_train)

RandomForestClassifier(n_jobs=-1)

In [129]:
clf.score(X_test, Y_test)

0.762120232708468

In [130]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.72      0.50      0.59      2654
    positive       0.78      0.90      0.83      5081

    accuracy                           0.76      7735
   macro avg       0.75      0.70      0.71      7735
weighted avg       0.76      0.76      0.75      7735

